In [114]:
import numpy as np

## Funciones varias

In [115]:
def Puerta(name):
    if name == "H":
        return 1/np.sqrt(2)*np.array([[1,1],[1,-1]],dtype=np.float128)
    if name == "swap":
        return np.array([[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]])
    if name == "cx":
        return np.array([[1,0,0,0],[0,0,0,1],[0,0,1,0],[0,1,0,0]])
    
    raise ValueError("Puerta no definida")

In [116]:
def swaps(distancia):
    qubits = (0,distancia)
    num_qubits = qubits[1] - qubits[0] + 1
    swap = np.eye(2**num_qubits)
    mask1 = 1 << qubits[0]
    #anti_mask = ((1 << num_qubits) - 1) ^ mask1
    mask2 = 1 << qubits[1]

    for i in range(2**num_qubits):
        if (i & mask1) and ((i & mask2) == 0):
            swap[i,i] = 0
            swap[i-mask1+mask2,i-mask1+mask2] = 0
            swap[i,i-mask1+mask2] = 1
            swap[i-mask1+mask2,i] = 1

    return swap

In [117]:
swaps(2)

array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]])

In [118]:
np.kron(Puerta("swap"),np.eye(2)) @ np.kron(np.eye(2),Puerta("swap")) @ np.kron(Puerta("swap"),np.eye(2))

array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]])

## Clase QRegistry

In [119]:
class QRegistry:
    def __init__(self,nqubits):
        self.nqubits = nqubits
        self.estado = np.append(np.array([1],dtype=complex),np.array([0 for _ in range(2**self.nqubits - 1)],dtype=complex)).reshape(-1,1)

    def ket(self):
        return self.estado
    
    def bra(self):
        return np.conjugate(np.transpose(self.estado))
    
    def M_densidad(self):
        return np.dot(self.ket(),self.bra())
    
    def aplicar_puerta(self, puerta, qubit):
        num_q_Puerta = int(np.log2(puerta.shape[0]))

        if qubit[0] < 0 or len(qubit) > self.nqubits:
            raise ValueError("Qubit imposible")
        
        elif num_q_Puerta > self.nqubits:
            raise ValueError("La puerta no cabe")
        
        cambios = None

        if len(qubit) > 1:
            posiciones = qubit + [j for j in range(self.nqubits) if j not in qubit]
            cambios = np.argsort(posiciones)
            ids = [posiciones[i] for i in cambios]
            # print(cambios, posiciones)

            for id in range(len(qubit)):
                target_id = np.where(cambios == id)[0][0]
                print(f"SWAP({id},{target_id})")
                if id != target_id:
                    self.aplicar_puerta(swaps(target_id - id),qubit=[id])

        operacion = np.kron(np.kron(np.eye(2**(self.nqubits - qubit[0] - num_q_Puerta)),puerta),np.eye(2**qubit[0]))

        self.estado = np.dot(operacion,self.ket())

        print(cambios)

        if cambios is not None:
            for id in range(len(qubit)):
                target_id = np.where(cambios == id)[0][0]
                print(f"DESSWAP({id},{target_id})")
                if id != target_id:
                    self.aplicar_puerta(swaps(target_id - id),qubit=[id])

        return self.estado
    
    def medir(self, qubit):
        if qubit < 0 or qubit >= self.nqubits:
            raise ValueError("Qubit imposible")
        
        lista = [j for j in range(2**self.nqubits) if j//2**(qubit)%2]
        # print(lista)
        r = np.random.rand()
        p = sum(np.absolute(self.estado[i,0])**2 for i in lista)

        if r < p:
            #print("Mide 1")
            listadg = [j for j in range(2**self.nqubits) if j//2**(qubit)%2 == 0]
            for i in listadg:
                self.estado[i,0] = 0
            self.estado = self.estado/np.sqrt(p)
        else:
            #print("Mide 0")
            for i in lista:
                self.estado[i,0] = 0
            self.estado = self.estado/np.sqrt(1-p)

        return int(r < p)


In [120]:
registro = QRegistry(4)
registro.aplicar_puerta(Puerta("H"),qubit=[0])
# registro.aplicar_puertaM(swaps(1),qubit=[0])
registro.aplicar_puerta(Puerta("cx"),qubit=[0,2])

None
SWAP(0,0)
SWAP(1,2)
None
[0 2 1 3]
DESSWAP(0,0)
DESSWAP(1,2)
None


array([[0.70710678+0.j],
       [0.        +0.j],
       [0.        +0.j],
       [0.        +0.j],
       [0.        +0.j],
       [0.70710678+0.j],
       [0.        +0.j],
       [0.        +0.j],
       [0.        +0.j],
       [0.        +0.j],
       [0.        +0.j],
       [0.        +0.j],
       [0.        +0.j],
       [0.        +0.j],
       [0.        +0.j],
       [0.        +0.j]], dtype=complex256)

## Ejemplos de uso

In [121]:
cx = Puerta("cx")
H = Puerta("H")

par_epr = QRegistry(2)
par_epr.aplicar_puerta(H,[0])
# display(par_epr.ket())

par_epr.aplicar_puerta(cx,[0,1])

display(par_epr.ket())

print(par_epr.medir(0))

print(par_epr.medir(1))

None
SWAP(0,0)
SWAP(1,1)
[0 1]
DESSWAP(0,0)
DESSWAP(1,1)


array([[0.70710678+0.j],
       [0.        +0.j],
       [0.        +0.j],
       [0.70710678+0.j]], dtype=complex256)

0
0


In [122]:
registro = QRegistry(3)
registro.aplicar_puerta(np.kron(Puerta("H"),Puerta("H")),[1])
registro.ket()

None


array([[0.5+0.j],
       [0. +0.j],
       [0.5+0.j],
       [0. +0.j],
       [0.5+0.j],
       [0. +0.j],
       [0.5+0.j],
       [0. +0.j]], dtype=complex256)

In [123]:
registro = QRegistry(3)
hadamard = 1/np.sqrt(2)*np.array([[1,1],[1,-1]],dtype=np.float128)
registro.aplicar_puerta(hadamard,[0])
registro.aplicar_puerta(hadamard,[1])
registro.aplicar_puerta(hadamard,[2])
registro.aplicar_puerta(hadamard,[0])
registro.aplicar_puerta(hadamard,[1])
registro.aplicar_puerta(hadamard,[2])

None
None
None
None
None
None


array([[1.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j]], dtype=complex256)

In [124]:
registro = QRegistry(3)
registro.medir(1)

0

In [125]:
ket0 = QRegistry(2)
ket0.ket()

array([[1.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j]])

In [126]:
ket0.bra()

array([[1.-0.j, 0.-0.j, 0.-0.j, 0.-0.j]])

In [127]:
ket0.M_densidad()

array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]])

In [128]:
ket0.aplicar_puerta(hadamard,qubit=[1])

None


array([[0.70710678+0.j],
       [0.        +0.j],
       [0.70710678+0.j],
       [0.        +0.j]], dtype=complex256)

In [129]:
ket0.medir(1)

0

In [130]:
ket0.ket()

array([[1.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j]], dtype=complex256)

In [131]:
registro = QRegistry(3)
registro.aplicar_puerta(hadamard,[0,1,2])
registro.ket()

SWAP(0,0)
SWAP(1,1)
SWAP(2,2)
[0 1 2]
DESSWAP(0,0)
DESSWAP(1,1)
DESSWAP(2,2)


array([[0.70710678+0.j],
       [0.70710678+0.j],
       [0.        +0.j],
       [0.        +0.j],
       [0.        +0.j],
       [0.        +0.j],
       [0.        +0.j],
       [0.        +0.j]], dtype=complex256)

# Coste de funciones

In [ ]:
'''
n = distancia
t(n) = 2 + 2 + 3 + 2^{2(n + 1)} + n + 1 + 1 + 3 + n + 2 + n + 1 + 
 + 2^{n+1}*(1 + 5 + {3 + 7 + 5 + 5, 0})

t(n) = 15 + 2^{2(n + 1)} + 3*n + 2^{n+1}*({26, 6})

O =  2^{2n}
w =  2^{2n}
theta = 2^{2n}
'''
def swaps(distancia):
    qubits = (0,distancia) # Coste 2 (asignacion + creacion)
    # n = distancia + 1
    num_qubits = qubits[1] - qubits[0] + 1 # Coste 2xO(1) + 3 (acceso a lista + operaciones + asignacion)
    swap = np.eye(2**num_qubits) # Coste 2^{2(distancia + 1)} + distancia + 1 + 1 (potencia y asignacion)
    mask1 = 1 << qubits[0] # Coste 1 (solo el desplazamiento) + 2 (asignacion + acceso)
    #anti_mask = ((1 << num_qubits) - 1) ^ mask1
    mask2 = 1 << qubits[1] # Coste distancia + 2

    for i in range(2**num_qubits): # Coste distancia + 1 + 1*iteracion
        if (i & mask1) and ((i & mask2) == 0): # Coste 5
            swap[i,i] = 0 # Coste 3
            swap[i-mask1+mask2,i-mask1+mask2] = 0 # Coste 7
            swap[i,i-mask1+mask2] = 1 # Coste 5
            swap[i-mask1+mask2,i] = 1 # Coste 5

    return swap

In [ ]:
'''
n = num_qubits
t(n) = 4 + (1 + (n + 1) + (8 + qubit)*2^{n}) + θ(1) + (2 + (7 + 2*θ(1))*2^{n-1}) + 
     + (2 + {1 + (n + 1) + (8 + qubit)*2^{n}, 1} + (5)*2^{n-1} + θ(1) + 1 + 2^{n}) + 3 = 
     = 13 + 2*θ(1) + n + (9 + qubit)*2^{n} + (12 + 2*θ(1))*2^{n-1} + {(n + 1) + (8 + qubit)*2^{n} , 0}

O(n) = 2^{n} # En el peor de los casos qubit = n - 1 => O(n) = n*2^{n}
w(n) = 2^{n}
θ(n) = 2^{n}
'''

def medir(self, qubit):
        if qubit < 0 or qubit >= self.nqubits: # 4 (asumimos que nunca entra dentro)
            raise ValueError("Qubit imposible")
        
        # lista = [] # 1
        # for j in range(2**self.nqubits): # (n + 1) + 2*2^{n}
        #   if j//2**(qubit)%2: # 1 + qubit + 2 + 2
        #       lista.append(j) # 1
        lista = [j for j in range(2**self.nqubits) if j//2**(qubit)%2]
        # len(lista) = len(listadg) = 2^{n-1}
        r = np.random.rand()    # O(1)
                                # w(1)
                                # θ(1)

        # p = 0 # 2 (interpretar como la llamada a la función sum?)
        # for i in lista: # (1 + θ(1))*iteracion = (1 + θ(1))*2^{n-1}
        #   p = p + np.absolute(self.estado[i,0])**2 # asignacion + potencia + suma + abs + acceso = 1 + 2 + 1 + θ(1) + 2 
        p = sum(np.absolute(self.estado[i,0])**2 for i in lista)

        if r < p: # 2
            #print("Mide 1")
            # lista = [] # 1
            # for j in range(2**self.nqubits): # (n + 1) + 2*2^{n}
            #   if j//2**(qubit)%2 == 0: # 1 + qubit + 2 + 2 + 1
            #       lista.append(j) # 1
            listadg = [j for j in range(2**self.nqubits) if j//2**(qubit)%2 == 0]

            for i in listadg: # 2*2^{n-1}
                self.estado[i,0] = 0 # 3
            self.estado = self.estado/np.sqrt(p) # θ(1) + 1 + 2^{n}
        else: # 1
            #print("Mide 0")
            for i in lista: # 2*2^{n-1}
                self.estado[i,0] = 0 # 3
            self.estado = self.estado/np.sqrt(1-p) # θ(1) + 1 + 2^{n}

        return int(r < p) # 3

# Coste espacial

In [ ]:
'''
Unidad = 32 bits
(Una variable que almacena un valor simple ocupa 1 espacio de memoria)

Memoria para calcular la medida (en el pico):

p(n) = 6 + 2^{n-1} + {2^{n-1}, 0} + 4*2^{n}

p(n) = O(2^{n})
p(n) = w(2^{n})
p(n) = θ(2^{n})

'''

def medir(self, qubit):
        # self -> 1
        # qubit -> 1
        if qubit < 0 or qubit >= self.nqubits:
            raise ValueError("Qubit imposible")
        
        # lista -> 2^{n-1}
        # j -> 1
        lista = [j for j in range(2**self.nqubits) if j//2**(qubit)%2]
        # j -> 0 (tras liberar memoria)
        # r -> 2 + coste(rand) = 2 + θ(1)
        r = np.random.rand()


        # p -> 1 + 2*2^{n-1} + 2 + θ(1)
        # p = sum([np.absolute(self.estado[i,0])**2 for i in lista])
        # p -> 1 + 2 + θ(1) (tras liberar memoria)
        # p -> 1 + 2 + θ(1)
        p = sum(np.absolute(self.estado[i,0])**2 for i in lista)

        if r < p:
            #print("Mide 1")
            del lista
            # lista -> 0 (tras liberar espacio)
            # lista -> 2^{n-1}
            lista = [j for j in range(2**self.nqubits) if j//2**(qubit)%2 == 0]

            # i -> 1
            for i in lista:
                self.estado[i,0] = 0
            # i -> 0 (tras liberar memoria)
            # lista -> 0 (tras liberar espacio)
            del lista
            # division -> 4*2^{n}
            self.estado = self.estado/np.sqrt(p)
            # division -> 0 (tras liberar memoria)
        else:
            #print("Mide 0")
            # i -> 1
            for i in lista:
                self.estado[i,0] = 0
            # i -> 0 (tras liberar memoria)
            # lista -> 0 (tras liberar espacio)
            del lista
            # division -> 4*2^{n}
            self.estado = self.estado/np.sqrt(1-p)
            # division -> 0 (tras liberar memoria)

        return int(r < p)